In [4]:
import re
import numpy as np
import pandas as pd
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from transformers import AutoTokenizer
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel

In [5]:
df = pd.read_csv(r"C:\Users\nunez\Facultad\Factor Data\Proyecto NLP canciones\Datasets Scrapeados\Folklore.csv")
df.dropna(subset="Lyrics",inplace=True)
def clean(text):
    cleaned_text = re.sub(r"^.*?Lyrics", "", text)
    cleaned_text = re.sub(r'\[.*?\]', '', cleaned_text)
    cleaned_text = re.sub(r'\([^)]*\)', '', cleaned_text)
    cleaned_text = re.sub(r'\\n', ' ', cleaned_text)
    cleaned_text = re.sub(r'\n', ' ', cleaned_text)
    cleaned_text = re.sub(r'Embed', '', cleaned_text)
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)
    cleaned_text = re.sub(r'you might also like', '', cleaned_text)
    cleaned_text = cleaned_text.lower()
    return cleaned_text
df["Lyrics"] = df["Lyrics"].astype(str).apply(clean)
df["Artist_genres"] = df["Artist_genres"].replace("[]",np.nan)
df.dropna(subset="Artist_genres",inplace=True)

In [6]:
df["Lyrics"]

0       despierta ya mujer asi yo puedo ver la mirada ...
4       cuando salí de santiago todo el camino lloré l...
9       dejé mi tierra cantora por conocer otros pagos...
12      en pampa de los huanacos yo vine dejando una f...
13      las nostalgias del ayer que traigo en mi canto...
                              ...                        
4575    formula denuncia  ministerio público de la nac...
4581    so who do you no tonigh lazy and gentleman the...
4583     dicen que el mundo es redondo pero tiene cuat...
4587    formula denuncia  ministerio público de la nac...
4589    formula denuncia  ministerio público de la nac...
Name: Lyrics, Length: 1781, dtype: object

In [7]:
from transformers import AutoTokenizer, AutoModel
import torch


model_name = "dccuchile/bert-base-spanish-wwm-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


sentence = df["Lyrics"].iloc[10]


tokens = tokenizer(sentence, return_tensors="pt")


with torch.no_grad():
    outputs = model(**tokens)


token_embeddings = outputs.last_hidden_state


embedding_for_first_token = token_embeddings[0]


all_token_embeddings = token_embeddings[0]
print(all_token_embeddings)


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[ 0.7545, -0.6453,  1.1431,  ..., -0.0022,  0.4830, -0.6728],
        [ 0.5292, -0.6151,  0.3465,  ..., -0.8031, -0.7017, -0.8839],
        [-0.0781, -1.1888,  0.8119,  ..., -0.7079, -0.3577, -1.6543],
        ...,
        [-0.2149, -0.0541, -0.5856,  ..., -0.2377, -0.3910, -1.2556],
        [-0.4785, -0.6545, -0.4262,  ...,  0.3892,  0.2259, -0.9087],
        [ 0.4094, -0.7337,  0.3785,  ...,  0.2436,  0.4474, -0.8795]])


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "dccuchile/bert-base-spanish-wwm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)




all_embeddings = []

for doc in df["Lyrics"]:


    tokens = tokenizer(doc, padding=True, truncation=True, return_tensors="pt")


    with torch.no_grad():
        outputs = model(**tokens)
        embeddings = outputs.last_hidden_state


    sentence_embeddings = torch.mean(embeddings, dim=1)

    all_embeddings.append(sentence_embeddings)


final_embeddings = torch.cat(all_embeddings, dim=0)




Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
final_embeddings

In [ ]:
v